# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
from logging import warning

import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from keras.src.layers import GlobalAveragePooling2D
from scipy.cluster.vq import kmeans
from sklearn.decomposition import PCA
from tensorboard.plugins.histogram.summary import histogram
from webencodings import labels

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
image_list=[]
path="newdata"
for filename in os.listdir(path):
    image=cv2.imread(os.path.join(path,filename))
    if image is not None:
        image=cv2.bitwise_not(image)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image_list.append(image_rgb)
print(len(image_list))

500


In [7]:
import cv2
import os

def face_segmentation(image):
    ycbcr_image=cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
    _, Cr, _ = cv2.split(ycbcr_image)    
    _, mask=cv2.threshold(Cr, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    segmented_image=cv2.bitwise_and(image, image, mask=mask)

    return segmented_image


In [8]:

segmented_images_list=[face_segmentation(image) for image in image_list]

In [9]:
def apply_morphology(segmented_image):
    kernel=np.ones((5, 5), np.uint8) 
    morphed_image=cv2.morphologyEx(segmented_image, cv2.MORPH_CLOSE, kernel)
    return morphed_image

def segment_and_process_images(image_list):
    segmented_images=[]
    for image in image_list:
        segmented_image=face_segmentation(image) 
        morphed_image=apply_morphology(segmented_image)  
        segmented_images.append(morphed_image)
    return segmented_images
segmented_images_list=segment_and_process_images(segmented_images_list)

In [10]:
import dlib
detector=dlib.get_frontal_face_detector()
predictor=dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
def get_face_landmarks(image):
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces=detector(gray)
    if len(faces) == 0:
        return None 
    for face in faces:
        landmarks=predictor(gray, face)
        landmarks_points=[]
        for n in range(68):
            x=landmarks.part(n).x
            y=landmarks.part(n).y
            landmarks_points.append((x, y))
        return np.array(landmarks_points)
    return None
def extract_geometric_features(landmarks):
    features=[]
    left_eye_center=np.mean(landmarks[36:42], axis=0) 
    right_eye_center=np.mean(landmarks[42:48], axis=0)
    eye_distance=np.linalg.norm(left_eye_center - right_eye_center)
    features.append(eye_distance)

    nose_tip=landmarks[30]
    eye_center_line=right_eye_center - left_eye_center
    nose_vector=nose_tip - left_eye_center
    angle=np.arctan2(nose_vector[1], nose_vector[0]) - np.arctan2(eye_center_line[1], eye_center_line[0])
    angle=np.degrees(angle)
    features.append(angle)
    
    chin=landmarks[8] 
    mouth_left=landmarks[48]
    mouth_right=landmarks[54]
    mouth_width=np.linalg.norm(mouth_left - mouth_right)
    features.append(mouth_width)
    
    chin_to_nose=np.linalg.norm(chin - nose_tip)
    features.append(chin_to_nose)
    return np.array(features)

def extract_landmark_features(image_list):
    geometric_features_list=[]
    for image in image_list:
        landmarks=get_face_landmarks(image)
        if landmarks is not None:
            geometric_features=extract_geometric_features(landmarks)
            geometric_features_list.append(geometric_features)
        else:
            geometric_features_list.append(np.zeros(4))
    
    return np.array(geometric_features_list)
def combine_features(original_features, geometric_features):
    combined_features=np.hstack((original_features, geometric_features))
    return combined_features


In [11]:
geometric_features=extract_landmark_features(image_list)

In [12]:
from skimage.feature import hog


def hog_extraction(image, orientations=8, pixels_per_cell=(8, 8),
                         cells_per_block=(2, 2), visualize=True):
    image_gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    fd, hog_image=hog(image_gray, orientations=orientations, pixels_per_cell=pixels_per_cell,visualize=visualize)
    return fd, hog_image

In [41]:
from keras.src.applications.vgg16 import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
def create_model_for_vgg():
    base=VGG16(weights='imagenet',include_top=False)
    for layer in base.layers[:15]: 
        layer.trainable = False
    v=base.output
    v=GlobalAveragePooling2D()(v)
    v=Dense(256,activation='relu')(v)
    v=Dropout(0.5)(v)
    v=Dense(128,activation='relu')(v)
    new_model=Model(inputs=base.input,outputs=v)
    return new_model

In [42]:
def vgg_feature_extraction(list):
    model=create_model_for_vgg()
    feautre_list=[]
    for image in list:
        resized_image=cv2.resize(image,(224,224))
        data=np.expand_dims(resized_image,axis=0)
        features=model.predict(data)
        feautre_list.append(features.flatten())
    return feautre_list

In [43]:
from keras.src.applications.resnet import ResNet50
def create_model_for_resnet():
    base=ResNet50(weights='imagenet',include_top=False)
    for layer in base.layers[:143]:
        layer.trainable = False
    b=base.output
    b=GlobalAveragePooling2D()(b)
    b=Dense(256,activation='relu')(b)
    b=Dropout(0.5)(b)
    b=Dense(128,activation='relu')(b)
    new_model=Model(inputs=base.input,outputs=b)
    return new_model

In [44]:
def resnet_feature_extraction(list):
    model=create_model_for_resnet()
    feautre_list=[]
    for image in list:
        resized_image=cv2.resize(image,(224,224))
        data=np.expand_dims(resized_image,axis=0)
        features=model.predict(data)
        feautre_list.append(features.flatten())
    return feautre_list    
    

In [17]:

from skimage.feature import local_binary_pattern
def lbp_extraction(image):
    image_gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp=local_binary_pattern(image_gray,P=8,R=1,method='uniform')
    bin_number=int(lbp.max()+1)
    #histogram,_=np.histogram(image,bins=bin_number)
    histogram,_=np.histogram(lbp.ravel(),bins=bin_number,range=(0,bin_number),density=True)
    return histogram,lbp

In [ ]:
##CLUSTERING

In [18]:
from sklearn.cluster import KMeans, DBSCAN
def kmeans_clustering(cluster_number,feature):
    kmeans=KMeans(n_clusters=cluster_number,random_state=42).fit(feature)
    return kmeans.labels_

    

In [19]:
def DBSCAN_clustering(feature):
    eps=0.5
    minimum_samples=5
    db=DBSCAN(eps=eps,min_samples=minimum_samples)
    labels = db.fit_predict(feature)
    return labels

In [86]:
def DBSCAN_clustering_for_nongeometric_or_combined(feature,epsilon,min_sample):
    dbscan=DBSCAN(eps=epsilon,min_samples=min_sample)
    labels=dbscan.fit_predict(feature)
    return labels

In [20]:
from keras import Input
def dec_input_with_autoencoder(dimension_of_input, dimension_of_encoding):
    layer=Input(shape=(dimension_of_input,))
    part_of_encoded=Dense(dimension_of_encoding, activation='relu')(layer)
    part_of_decoded=Dense(dimension_of_input, activation='sigmoid')(part_of_encoded)
    autoencoder=Model(layer,part_of_decoded)    
    autoencoder.compile(optimizer='adam', loss='mse')
    encoder=Model(layer,part_of_encoded)
    return autoencoder,encoder

In [21]:
def dec_clustering(feautre, cluster_number):
    feautre=np.array(feautre) 
    dimension_of_input=feautre.shape[1]
    dimension_of_encoding=128
    autoencoder,encoder=dec_input_with_autoencoder(dimension_of_input,dimension_of_encoding)
    autoencoder.fit(feautre, feautre, epochs=50, batch_size=256, shuffle=True,verbose=1)
    encoding_part=encoder.predict(feautre)
    kmeans=KMeans(n_clusters=cluster_number, random_state=42)
    labels=kmeans.fit_predict(encoding_part)
    return labels 

In [31]:
from sklearn.metrics import silhouette_score
def silhoutte_evaluation(features,labels):
    success_score=silhouette_score(features,labels)
    return success_score

In [ ]:
##TESTING

In [72]:
print("KMeans Clustering With Only Geometric Features")
for iteration in range(2,6):
    kmeans_only_geometric_features=kmeans_clustering(iteration,geometric_features)
    silhoutte_score = silhoutte_evaluation(geometric_features,kmeans_only_geometric_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")


KMeans Clustering With Only Geometric Features
For cluster number 2, Silhouette score: 0.9709011229759201
For cluster number 3, Silhouette score: 0.9299818826767892
For cluster number 4, Silhouette score: 0.917907574725521
For cluster number 5, Silhouette score: 0.9176820261511776


In [73]:
print("Deep Embedded Clustering With Only Geometric Features")
for iteration in range(2,6):
    dec_only_geometric_features=dec_clustering(geometric_features,iteration)
    silhoutte_score = silhoutte_evaluation(geometric_features,dec_only_geometric_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")


Deep Embedded Clustering With Only Geometric Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 138.5256 
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 135.8200 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 135.5142 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 136.5215 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 132.0957 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 129.2132 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 128.7084 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 125.4417 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 139.3669 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 140.4995 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 132.6398 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 136.1250 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 127.2514 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 137.1744 
Epoch 15/50
2/2 

In [74]:
print("DBSCAN Clustering With Only Geometric Features")
dbscan_only_geometric_features=DBSCAN_clustering(geometric_features)
silhoutte_evaluation(geometric_features,dbscan_only_geometric_features)

DBSCAN Clustering With Only Geometric Features


0.9709011229759201

In [26]:
hog_features=[hog_extraction(image) for image in segmented_images_list]
lbp_features=[lbp_extraction(image) for image in segmented_images_list]

hog_features = [hog[0].flatten() for hog in hog_features]  
lbp_features = [lbp[0].flatten() for lbp in lbp_features]  



In [75]:
print("KMeans Clustering With Hog And Geometric Features")
combined_with_hog=combine_features(hog_features,geometric_features)
for iteration in range(2,6):
    kmeans_with_hog_and_geometric=kmeans_clustering(iteration,combined_with_hog)
    silhoutte_score = silhoutte_evaluation(combined_with_hog,kmeans_with_hog_and_geometric)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

KMeans Clustering With Hog And Geometric Features
For cluster number 2, Silhouette score: 0.8381403773553829
For cluster number 3, Silhouette score: 0.1368630388997004
For cluster number 4, Silhouette score: 0.09202845687636571
For cluster number 5, Silhouette score: 0.09053084153186339


In [76]:
print("KMeans Clustering With Lbp And Geometric Features")
combined_with_lbp=combine_features(lbp_features,geometric_features)
for iteration in range(2,6):
    kmeans_with_lbp_and_geometric=kmeans_clustering(iteration,combined_with_lbp)
    silhoutte_score = silhoutte_evaluation(combined_with_lbp,kmeans_with_lbp_and_geometric)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")


KMeans Clustering With Lbp And Geometric Features
For cluster number 2, Silhouette score: 0.9685547737701279
For cluster number 3, Silhouette score: 0.9274099251862749
For cluster number 4, Silhouette score: 0.9153137823891815
For cluster number 5, Silhouette score: 0.9149880487717929


In [77]:
print("Deep Embedded Clustering With Hog And Geometric Features")
for iteration in range(2,6):
    dec_with_hog_and_geometric=dec_clustering(combined_with_hog,iteration)
    silhoutte_score = silhoutte_evaluation(combined_with_hog,dec_with_hog_and_geometric)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")


Deep Embedded Clustering With Hog And Geometric Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - loss: 0.2709 
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.2620
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.2192
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.1596
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1120
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0934
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0896
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0897
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0926
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0882
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0877
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0879
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0887
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0850
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [78]:
print("Deep Embedded Clustering With Lbp And Geometric Features")
for iteration in range(2,6):
    dec_with_lbp_and_geometric=dec_clustering(combined_with_lbp,iteration)
    silhoutte_score = silhoutte_evaluation(combined_with_lbp,dec_with_lbp_and_geometric)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")


Deep Embedded Clustering With Lbp And Geometric Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 37.3578  
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 38.7640 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 38.2951 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 36.5418 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.6506 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39.6344 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 37.8146 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 39.0350 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.5142 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 34.8506 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 40.8496 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 39.0597 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 37.6568 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 36.5252 
Epoch 15/50
2/2 ━━━━━━━━━━━

In [79]:
print("KMeans Clustering With Only Lbp Features")
for iteration in range(2,6):
    kmeans_with_only_lbp=kmeans_clustering(iteration,lbp_features)
    silhoutte_score = silhoutte_evaluation(lbp_features,kmeans_with_only_lbp)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")


KMeans Clustering With Only Lbp Features
For cluster number 2, Silhouette score: 0.5412751464226153
For cluster number 3, Silhouette score: 0.48813324442288036
For cluster number 4, Silhouette score: 0.4706726465377566
For cluster number 5, Silhouette score: 0.45669972304911355


In [80]:
print("Deep Embedded Clustering With Only Lbp Features")
for iteration in range(2,6):
    dec_with_only_lbp=dec_clustering(lbp_features,iteration)
    silhoutte_score = silhoutte_evaluation(lbp_features,dec_with_only_lbp)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")


Deep Embedded Clustering With Only Lbp Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 0.1880  
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1850 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1818 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1789 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1753 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1724 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1691 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1657 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1625 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1594 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1557 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1520 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1485 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.1445 
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step -

In [81]:
print("KMeans Clustering With Only Hog Features")
for iteration in range(2,6):
    kmeans_with_only_hog=kmeans_clustering(iteration,hog_features)
    silhoutte_score = silhoutte_evaluation(hog_features,kmeans_with_only_hog)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

KMeans Clustering With Only Hog Features
For cluster number 2, Silhouette score: 0.07155945033911083
For cluster number 3, Silhouette score: 0.030955933191013633
For cluster number 4, Silhouette score: 0.022975757478195122
For cluster number 5, Silhouette score: 0.025650872704043964


In [82]:
print("Deep Embedded Clustering With Only Hog Features")
for iteration in range(2,6):
    dec_with_only_hog=dec_clustering(hog_features,iteration)
    silhoutte_score = silhoutte_evaluation(hog_features,dec_with_only_hog)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

Deep Embedded Clustering With Only Hog Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - loss: 0.1985 
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1915
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1629
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1096
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0535
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 0.0224
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0144
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0146
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0142
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.0150
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0145
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.0139
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.0145
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.0140
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step 

In [60]:
from sklearn.preprocessing import StandardScaler
def scale_features(features):
    scaler = StandardScaler()
    return scaler.fit_transform(features)



In [62]:
from sklearn.neighbors import NearestNeighbors
def find_optimal_eps(features):
    neighbors=NearestNeighbors(n_neighbors=5)
    neighbors_fit=neighbors.fit(features)
    distances, indices=neighbors_fit.kneighbors(features)
    distances=np.sort(distances[:, 4], axis=0)
    return distances

In [87]:
print("DBSCAN Clustering With Only Lbp Features")
scaled_features_lbp=scale_features(lbp_features)
distances=find_optimal_eps(scaled_features_lbp)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_only_lbp_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_lbp,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_lbp, dbscan_only_lbp_features)
print(f"Silhouette score: {silhoutte_score}")



DBSCAN Clustering With Only Lbp Features
Silhouette score: 0.2540010091809407


In [88]:
print("DBSCAN Clustering With Only Hog Features")
scaled_features_hog=scale_features(hog_features)
distances=find_optimal_eps(scaled_features_hog)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_only_hog_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_hog,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_hog, dbscan_only_hog_features)
print(f"Silhouette score: {silhoutte_score}")

DBSCAN Clustering With Only Hog Features
Silhouette score: 0.06820984642255164


In [89]:
print("DBSCAN Clustering With Lbp And Geometric Features")
scaled_features_lbp_combined=scale_features(combined_with_lbp)
distances=find_optimal_eps(scaled_features_lbp_combined)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_combined_lbp_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_lbp_combined,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_lbp_combined, dbscan_combined_lbp_features)
print(f"Silhouette score: {silhoutte_score}")


DBSCAN Clustering With Lbp And Geometric Features
Silhouette score: 0.4127767407304654


In [92]:
print("DBSCAN Clustering With Lbp And Geometric Features NonScaled")
dbscan_combined_lbp_features_non_scaled=DBSCAN_clustering(combined_with_lbp)
silhoutte_score=silhoutte_evaluation(combined_with_lbp,dbscan_combined_lbp_features_non_scaled)
print(f"Silhouette score: {silhoutte_score}")

DBSCAN Clustering With Lbp And Geometric Features NonScaled
Silhouette score: 0.9685547737701279


In [91]:
print("DBSCAN Clustering With Hog And Geometric Features")
scaled_features_hog_combined=scale_features(combined_with_hog)
distances=find_optimal_eps(scaled_features_hog_combined)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_combined_hog_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_hog_combined,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_hog_combined, dbscan_combined_hog_features)
print(f"Silhouette score: {silhoutte_score}")

DBSCAN Clustering With Hog And Geometric Features
Silhouette score: 0.06677754259371187


In [93]:
vgg_features=vgg_feature_extraction(segmented_images_list)
resnet_features=resnet_feature_extraction(segmented_images_list)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 571ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 179ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 159ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 160ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 164ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 228ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 183ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 

In [94]:
combined_with_vgg=combine_features(vgg_features,geometric_features)


In [96]:
print("KMeans Clustering With Only Vgg Features")
for iteration in range(2,6):
    kmeans_only_vgg_features=kmeans_clustering(iteration,vgg_features)
    silhoutte_score = silhoutte_evaluation(vgg_features,kmeans_only_vgg_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

KMeans Clustering With Only Vgg Features
For cluster number 2, Silhouette score: 0.12476536631584167
For cluster number 3, Silhouette score: 0.09709127247333527
For cluster number 4, Silhouette score: 0.08268306404352188
For cluster number 5, Silhouette score: 0.07498057186603546


In [97]:
print("KMeans Clustering With Vgg And Geometric Features")
for iteration in range(2,6):
    kmeans_combined_vgg_features=kmeans_clustering(iteration,combined_with_vgg)
    silhoutte_score = silhoutte_evaluation(combined_with_vgg,kmeans_combined_vgg_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

KMeans Clustering With Vgg And Geometric Features
For cluster number 2, Silhouette score: 0.7402158133036604
For cluster number 3, Silhouette score: 0.1847400923593762
For cluster number 4, Silhouette score: 0.15554928375602967
For cluster number 5, Silhouette score: 0.10865897068875058


In [98]:
print("Deep Embedded Clustering With Only Vgg Features")
for iteration in range(2,6):
    dec_only_vgg_features=dec_clustering(vgg_features,iteration)
    silhoutte_score = silhoutte_evaluation(vgg_features,dec_only_vgg_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

Deep Embedded Clustering With Only Vgg Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - loss: 4.9029
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.6880 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.5363 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 4.3413 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.2509 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.1492 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.0515 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.9977 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 3.8938 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 3.9099 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.9012 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.8504 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.8338 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.8291 
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - l

In [99]:
print("Deep Embedded Clustering With Vgg And Geometric Features")
for iteration in range(2,6):
    dec_combined_vgg_features=dec_clustering(combined_with_vgg,iteration)
    silhoutte_score = silhoutte_evaluation(combined_with_vgg,dec_combined_vgg_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

Deep Embedded Clustering With Vgg And Geometric Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 8.8760  
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.9252 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.5633 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.2919 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 7.9099 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.1068 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.8007 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.8603 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.7744 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.0870 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 8.4682 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5831 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.8356 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 7.5887 
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [100]:
print("DBSCAN Clustering With Only Vgg Features")
scaled_features_vgg=scale_features(vgg_features)
distances=find_optimal_eps(scaled_features_vgg)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_only_vgg_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_vgg,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_vgg, dbscan_only_vgg_features)
print(f"Silhouette score: {silhoutte_score}")

DBSCAN Clustering With Only Vgg Features
Silhouette score: 0.3511401258580817


In [101]:
print("DBSCAN Clustering With Vgg And Geometric Features")
scaled_features_vgg_combined=scale_features(combined_with_vgg)
distances=find_optimal_eps(scaled_features_vgg_combined)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_combined_vgg_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_vgg_combined,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_vgg_combined, dbscan_combined_vgg_features)
print(f"Silhouette score: {silhoutte_score}")

DBSCAN Clustering With Vgg And Geometric Features
Silhouette score: 0.3327370804512986


In [102]:
combined_with_res=combine_features(resnet_features,geometric_features)

In [103]:
print("KMeans Clustering With Only Resnet Features")
for iteration in range(2,6):
    kmeans_only_resnet_features=kmeans_clustering(iteration,resnet_features)
    silhoutte_score = silhoutte_evaluation(resnet_features,kmeans_only_resnet_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

KMeans Clustering With Only Resnet Features
For cluster number 2, Silhouette score: 0.11007101833820343
For cluster number 3, Silhouette score: 0.08310391753911972
For cluster number 4, Silhouette score: 0.08401626348495483
For cluster number 5, Silhouette score: 0.07150421291589737


In [104]:
print("KMeans Clustering With Resnet And Geometric Features")
for iteration in range(2,6):
    kmeans_combined_resnet_features=kmeans_clustering(iteration,combined_with_res)
    silhoutte_score = silhoutte_evaluation(combined_with_res,kmeans_combined_resnet_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

KMeans Clustering With Resnet And Geometric Features
For cluster number 2, Silhouette score: 0.8941279952284905
For cluster number 3, Silhouette score: 0.1717889519482525
For cluster number 4, Silhouette score: 0.12847799344926564
For cluster number 5, Silhouette score: 0.10309917663817833


In [105]:
print("Deep Embedded Clustering With Only Resnet Features")
for iteration in range(2,6):
    dec_only_resnet_features=dec_clustering(resnet_features,iteration)
    silhoutte_score = silhoutte_evaluation(resnet_features,dec_only_resnet_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

Deep Embedded Clustering With Only Resnet Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.5476  
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.5086 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4742 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4411 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.4096 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3827 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.3553 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3317 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.3077 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2932 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2806 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.2682 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.2620 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.2573 
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste

In [106]:
print("Deep Embedded Clustering With Resnet And Geometric Features")
for iteration in range(2,6):
    dec_combined_resnet_features=dec_clustering(combined_with_res,iteration)
    silhoutte_score = silhoutte_evaluation(combined_with_res,dec_combined_resnet_features)
    print(f"For cluster number {iteration}, Silhouette score: {silhoutte_score}")

Deep Embedded Clustering With Resnet And Geometric Features
Epoch 1/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.5786  
Epoch 2/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.6482 
Epoch 3/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.5133 
Epoch 4/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.4420 
Epoch 5/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.3541 
Epoch 6/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.3694 
Epoch 7/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.9864 
Epoch 8/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 4.1701 
Epoch 9/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.3167 
Epoch 10/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 4.2257 
Epoch 11/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.8583 
Epoch 12/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4.4640 
Epoch 13/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3.7545 
Epoch 14/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3.9024 
Epoch 15/50
2/2 ━━━━━━━━━━━━━━━━━━━━ 0

In [107]:
print("DBSCAN Clustering With Only Resnet Features")
scaled_features_resnet=scale_features(resnet_features)
distances=find_optimal_eps(scaled_features_resnet)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_only_resnet_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_resnet,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_resnet, dbscan_only_resnet_features)
print(f"Silhouette score: {silhoutte_score}")

DBSCAN Clustering With Only Resnet Features
Silhouette score: 0.3351542245867766


In [108]:
print("DBSCAN Clustering With Resnet And Geometric Features")
scaled_features_resnet_combined=scale_features(combined_with_res)
distances=find_optimal_eps(scaled_features_resnet_combined)
optimal_eps = distances[int(len(distances) * 0.9)]
dbscan_combined_resnet_features=DBSCAN_clustering_for_nongeometric_or_combined(scaled_features_resnet_combined,optimal_eps,5)
silhoutte_score = silhoutte_evaluation(scaled_features_resnet_combined, dbscan_combined_resnet_features)
print(f"Silhouette score: {silhoutte_score}")

DBSCAN Clustering With Resnet And Geometric Features
Silhouette score: 0.3300665357064164


In [ ]:
#-----------